# Introduction
In this notebook, we will demonstrate the classification of EEG based Brain-Computer Interface data from a SSVEP experiment.

Two classification methods impletemented in the open source [BCI-Baseline](https://github.com/okbalefthanded/BCI-Baseline) library are evaluated on
a single subject data taken from the Tsinghua SSVEP benchmark [1].

## Methods:
* Canonical Correlation Analysis [2]

* Task-Related Component Analysis [3]

## Data : 
--

**References**

[1] Y. Wang, X. Chen, X. Gao, S. Gao, A Benchmark Dataset for SSVEP-Based 
    Brain-Computer Interfaces, IEEE Trans. Neural Syst. Rehabil. 
    Eng. 4320 (2016) 1–1. doi:10.1109/TNSRE.2016.2627556

[2] Z. Lin, C. Zhang, W. Wu, and X. Gao, “Frequency Recognition Based on Canonical Correlation Analysis for SSVEP-Based BCIs,” vol. 53, no. 12, pp. 2610–2614, 2006.

[3] M. Nakanishi, Y. Wang, X. Chen, Y. -T. Wang, X. Gao, and T.-P. Jung,
    "Enhancing detection of SSVEPs for a high-speed brain speller using
    task-related component analysis", IEEE Trans. Biomed. Eng, 65(1): 104-112, 2018.
    http://ieeexplore.ieee.org/document/7904641/

# 1. Install and import packages


In [1]:
!pip install git+https://github.com/okbalefthanded/BCI-Baseline.git

  Cloning https://github.com/okbalefthanded/BCI-Baseline.git to /tmp/pip-req-build-azvm9hpr
  Running command git clone -q https://github.com/okbalefthanded/BCI-Baseline.git /tmp/pip-req-build-azvm9hpr
  Created wheel for BCI-baseline: filename=BCI_baseline-0.1.0-py3-none-any.whl size=16435 sha256=7866dab76ec2cd9885c836743f7762a82d6b9293d4fe57784313cf903ee106a5
  Stored in directory: /tmp/pip-ephem-wheel-cache-_c4ufdro/wheels/b1/11/90/8fea6b75837cbc822655a5526c89cd941488734faa41d677ff
Successfully built BCI-baseline


In [2]:
import pandas as pd
import numpy as np
import random
# 
from baseline.ssvep.cca import CCA
from baseline.ssvep.trca import TRCA
from sklearn.metrics import accuracy_score
from scipy.io import loadmat
# set seeds
np.random.seed(42)
random.seed(42)

# 2. Download Data

In [3]:
!wget -r --no-parent ftp://anonymous@sccn.ucsd.edu/pub/ssvep_benchmark_dataset/S27.mat

--2021-08-02 19:09:30--  ftp://anonymous@sccn.ucsd.edu/pub/ssvep_benchmark_dataset/S27.mat
           => ‘sccn.ucsd.edu/pub/ssvep_benchmark_dataset/.listing’
Resolving sccn.ucsd.edu (sccn.ucsd.edu)... 169.228.38.2
Connecting to sccn.ucsd.edu (sccn.ucsd.edu)|169.228.38.2|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/ssvep_benchmark_dataset ... done.
==> PASV ... done.    ==> LIST ... done.

sccn.ucsd.edu/pub/s     [ <=>                ]   2.78K  --.-KB/s    in 0s      

2021-08-02 19:09:31 (5.66 MB/s) - ‘sccn.ucsd.edu/pub/ssvep_benchmark_dataset/.listing’ saved [2850]

Removed ‘sccn.ucsd.edu/pub/ssvep_benchmark_dataset/.listing’.
--2021-08-02 19:09:31--  ftp://anonymous@sccn.ucsd.edu/pub/ssvep_benchmark_dataset/S27.mat
           => ‘sccn.ucsd.edu/pub/ssvep_benchmark_dataset/S27.mat’
==> CWD not required.
==> PASV ... done.    ==> RETR S27.mat ... done.
Length: 105744153 (101M)

sccn.ucsd.edu/pub/s

In [4]:
folder = 'sccn.ucsd.edu/pub/ssvep_benchmark_dataset'

In [25]:
# (1: Pz, 2: PO5,3:  PO3, 4: POz, 5: PO4, 6: PO6, 7: O1, 8: Oz, and 9: O2)
ch_names=[ 'FP1', 'FPZ', 'FP2', 'AF3', 'AF4', 'F7', 'F5', 'F3',
           'F1', 'FZ', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5',
            'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8', 'T7',
            'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8', 'M1',
            'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6',
            'TP8', 'M2', 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2',
            'P4', 'P6', 'P8', 'PO7', 'PO5', 'PO3', 'POz', 'PO4',
              'PO6', 'PO8', 'CB1', 'O1', 'Oz', 'O2', 'CB2']
chs = [47,53, 54,55,56,57,60,61,62]
[ch_names[c] for c in chs]

['PZ', 'PO5', 'PO3', 'POz', 'PO4', 'PO6', 'O1', 'Oz', 'O2']

In [26]:
data = loadmat(f"{folder}/S27.mat")
fs = 250
eeg = data['data'].transpose((1, 0, 2, 3))
eeg = eeg[159:284, chs, :, :] # 500 ms from cue onset
samples, channels, targets, blocks = eeg.shape
y_train = np.tile(np.arange(1, targets + 1), (1, blocks-1)).squeeze()
y_test = np.arange(1, targets + 1)

# 3. Evaluation


In [56]:
def loo_evaluation(eeg, y_train, y_test, clf):
  accs = []
  samples, channels, targets, blocks = eeg.shape
  for loocvi in range(blocks):    
    x_train = np.delete(eeg, loocvi, axis=-1).reshape((samples, channels, blocks-1*targets), order='F')
    x_test = eeg[:,:,:,loocvi]    
    clf = clf.fit(x_train, y_train)
    p1 = clf.predict(x_train, y_train) 
    p2 = clf.predict(x_test, y_test)
    train_acc = accuracy_score(y_train, p1+1)
    test_acc = accuracy_score(y_test, p2+1)
    accs.append(test_acc)
    print(f'train acc {train_acc*100:.2f} % test acc {test_acc*100:.2f}% ')
  print(f'Mean accuracy {np.array(accs).mean().round(3)*100} % ')

# 3.1 CCA

In [55]:
frequencies=[8., 9., 10., 11., 12., 13., 14., 15., 8.2, 9.2,
            10.2, 11.2, 12.2, 13.2, 14.2, 15.2, 8.4, 9.4, 10.4, 11.4,
            12.4, 13.4, 14.4, 15.4, 8.6, 9.6, 10.6, 11.6, 12.6, 13.6,
              14.6, 15.6, 8.8, 9.8, 10.8, 11.8, 12.8, 13.8, 14.8, 15.8]

phase=[0., 1.57079633, 3.14159265, 4.71238898, 0.,
      1.57079633, 3.14159265, 4.71238898, 1.57079633, 3.14159265,
      4.71238898, 0., 1.57079633, 3.14159265, 4.71238898,
      0., 3.14159265, 4.71238898, 0., 1.57079633,
      3.14159265, 4.71238898, 0., 1.57079633, 4.71238898,
      0., 1.57079633, 3.14159265, 4.71238898, 0.,
      1.57079633, 3.14159265, 0., 1.57079633, 3.14159265,
      4.71238898, 0., 1.57079633, 3.14159265, 4.71238898]
      
clf = CCA(n_harmonics=2, frequencies=frequencies, length=samples/fs)
X = eeg.reshape((samples, channels, blocks*targets), order='F').transpose((1,0,2))
Y = np.concatenate((y_train, y_test))
clf.fit(X)
pr = []
for tr in range(X.shape[2]):
      pr.append(clf.predict(X[:,:,tr])+1)
acc = accuracy_score(Y, pr)
    
print(f'CCA Accuracy:  {acc*100:.2f} %')

CCA Accuracy:  14.58 %


# 3.2 Task-Related Component Analysis

## 3.2.1 TRCA

In [57]:
clf =  TRCA(5, False, fs)
loo_evaluation(eeg, y_train, y_test, clf)

train acc 100.00 % test acc 72.50% 
train acc 100.00 % test acc 92.50% 
train acc 100.00 % test acc 75.00% 
train acc 100.00 % test acc 82.50% 
train acc 100.00 % test acc 80.00% 
train acc 100.00 % test acc 77.50% 
Mean accuracy 80.0 % 


## 3.2.2 ensemble TRCA

In [58]:
clf =  TRCA(5, True, fs)
loo_evaluation(eeg, y_train, y_test, clf)

train acc 100.00 % test acc 87.50% 
train acc 100.00 % test acc 97.50% 
train acc 100.00 % test acc 100.00% 
train acc 100.00 % test acc 95.00% 
train acc 100.00 % test acc 87.50% 
train acc 100.00 % test acc 92.50% 
Mean accuracy 93.30000000000001 % 
